# Gerando os dados de raça e cor

Os dados de raça e cor são gerados utilizando o método criado pelo [http://patadata.org](http://patadata.org).

Primeio é necessário fazer o download dos seguintes dados:

- Polígonos de setores censitários, usando o arquivo [download_setores.ipynb](./download_setores.ipynb), especificando na variável correta o ano de `2022`.
- Dados do censo de 2022, usando o arquivo [download_censo.ipynb](./download_setores.ipynb), especificando o ano de `2022` na variável correta.


## Importando bibliotecas necessárias

In [2]:
# Importing libraries

import pandas as pd
import geopandas as gpd
from os import path, makedirs
from glob import glob
from shapely.geometry import Point, MultiPoint
import numpy as np
from estudos_tematicos.globalmaptiles import GlobalMercator
from tqdm import tqdm

## Informações necessárias para a execução do código 

In [4]:
# Defining the paths
# pasta onde estão os dados de cor e raça
base_path = "./COR_RACA/2022"
csv_folder = path.join(base_path, "CSVS")  # pasta onde estão os arquivos csv (censo)
rand_folder = path.join(
    base_path, "pontos_aleatorios"
)  # pasta onde serão salvos os arquivos de pontos aleatórios
makedirs(rand_folder, exist_ok=True)

# Pasta dos setores censitários
setor_sensitario_folder = path.join(".", "SETORES_CENSITARIOS", "2022", "gpkg")

agregado_file = next(
    iter(glob(path.join(csv_folder, "*.csv")))
)  # arquivo de dados agregados do censo
states = [
    "AC",
    "AL",
    "AM",
    # "AP",
    # "BA",
    "CE",
    # "DF",
    # "ES",
    # "GO",
    # "MA",
    "MG",
    "MS",
    "MT",
    "PA",
    "PB",
    # "PE", #! Parrou na organização do arquivo
    "PI",
    "PR",
    "RJ",
    "RN",
    "RO",
    # "RR",
    "RS",
    "SC",
    # "SE",
    # "SP", #! Parrou na organização do arquivo
    # "TO",
    # "SP",
]  # estados que serão processados

states = ["TO", "SE", "RO", "PA", "PR", "MD"]

# Campos de interesse do arquivo de censo, com seus respectivos nomes
pessoa_fields = {
    "V01317": "pessoas_brancas",
    "V01318": "pessoas_pretas",
    "V01319": "pessoas_amarelas",
    "V01320": "pessoas_pardas",
    "V01321": "pessoas_indigenas",
    "CD_SETOR": "CD_SETOR",
}

# Tipo dos campos do arquivo de censo
pessoa_fields_dypes = {
    "V01317": "Int64",
    "V01318": "Int64",
    "V01319": "Int64",
    "V01320": "Int64",
    "V01321": "Int64",
    "CD_SETOR": "object",
}

# Campos de interesse do arquivo de setores censitários com seus respectivos nomes
setor_fiels = {"v0001": "total_pessoas"}

# Campos do setor censitário que serão utilizados no processo
setor_keep_fields = [
    "CD_SETOR",
    "SITUACAO",
    "CD_SIT",
    "CD_TIPO",
    "AREA_KM2",
    "CD_REGIAO",
    "NM_REGIAO",
    "CD_UF",
    "NM_UF",
    "CD_MUN",
    "NM_MUN",
    "CD_DIST",
    "NM_DIST",
    "CD_SUBDIST",
    "NM_SUBDIST",
    "CD_BAIRRO",
    "NM_BAIRRO",
    "CD_NU",
    "NM_NU",
    "CD_FCU",
    "NM_FCU",
    "CD_AGLOM",
    "NM_AGLOM",
    "CD_RGINT",
    "NM_RGINT",
    "CD_RGI",
    "NM_RGI",
    "CD_CONCURB",
    "NM_CONCURB",
    "v0001",
    "geometry",
]

# Campos de raça e cor e o label que será usado nos pontos
race_fields = {
    "pessoas_brancas": "Brancos",
    "pessoas_pretas": "Pretos",
    "pessoas_amarelas": "Amarelas",
    "pessoas_pardas": "Pardos",
    "pessoas_indigenas": "Indígenas",
}

# Campos que serão mantidos no arquivo final de pontos
points_cols = [
    "CD_SETOR",
    "SITUACAO",
    "CD_SIT",
    "CD_TIPO",
    "AREA_KM2",
    "CD_REGIAO",
    "NM_REGIAO",
    "CD_UF",
    "NM_UF",
    "CD_MUN",
    "NM_MUN",
    "CD_DIST",
    "NM_DIST",
    "CD_SUBDIST",
    "NM_SUBDIST",
    "CD_BAIRRO",
    "NM_BAIRRO",
    "CD_NU",
    "NM_NU",
    "CD_FCU",
    "NM_FCU",
    "CD_AGLOM",
    "NM_AGLOM",
    "CD_RGINT",
    "NM_RGINT",
    "CD_RGI",
    "NM_RGI",
    "CD_CONCURB",
    "NM_CONCURB",
    "total_pessoas",
    *race_fields.keys(),
    "raca_cor",
    "tile_x",
    "tile_y",
    "quadkey",
    "geometry",
]

raca_cor_poligono = path.join(".", "SETORES_CENSITARIOS", "2022", "gpkg", "raca_cor_pol.gpkg")

## Fuções necessárias para a execução do código

In [5]:

merc = GlobalMercator()


# Função para gerar N pontos aleatórios dentro de uma geometria
def generate_random_points(geometry, num_points):
    minx, miny, maxx, maxy = geometry.bounds
    points = []
    while len(points) < num_points:
        random_x = np.random.uniform(minx, maxx, num_points)
        random_y = np.random.uniform(miny, maxy, num_points)
        candidate_points = MultiPoint([Point(x, y) for x, y in zip(random_x, random_y)])
        # Filtra apenas os pontos dentro da geometria
        points_within = [p for p in candidate_points.geoms if geometry.contains(p)]
        points.extend(points_within)
    return points[:num_points]

def filtra_arquvios_sigla_estados(file_list: list[str], states: list[str]) -> list:
    """Filtra a lista de arquivos de acordo com a sigla dos estados

    Arguments:
        file_list {list} -- Lista de arquivos
        states {list} -- Lista de siglas dos estados

    Returns:
        list -- Lista de arquivos filtrada
    """
    return [
        file
        for file in file_list
        if path.basename(file).split("_")[0].upper() in states
    ]


## Fazendo a união das tabelas e gerando os pontos

No processo do [patadata.org](http://patadata.org) o processo de união dos dados era feito em um software de GIS, como o QGIS, para depois termos os pontos gerados. Aqui foi feito via código utilizando a biblioteca [pandas](https://pandas.pydata.org), para trabalhar os dados tabulares, em conjunto com a biblioteca [geopandas](https://geopandas.org) para trabalhar os dados geográficos.

Esse diferencial permite gerar os arquivos de forma mais automática, com o mínimo de intevenção manual.

Também foi feita a atualização do código do repositório para o [Python](www.python.org) mais recente, pois o código antigo não era compatível com as versões 3.xx do mesmo.

Além disso, a criação dos pontos foi incorporado ao processo, gerando o arquivo de pontos logo em seguida a união.

Abrindo o arquivo de agregados pegando apenas os campos necessários:

In [6]:
pessoa_df = pd.read_csv(
    agregado_file,
    sep=";",
    encoding="latin1",
    usecols=[*pessoa_fields.keys()],  # * abrimos apenas as colunas que precisamos
    na_values=["X"],  # * aqui estamos transformando em x em NaN/None
    dtype=pessoa_fields_dypes,  # * aqui estamos informando os tipos corretos dos campos
)

pessoa_df.rename(columns=pessoa_fields, inplace=True) # * renomeando as colunas

pessoa_cols = pessoa_df.columns

# Trabalhando os setores sensitários

Aqui vamos abrir cada um dos arquivos de setores censitários que baixamos anteriormente, renomear o campo `v0001` para `total_pessoas`.
Em seguida iremos fazer a união dos dados do censo de 2022 (pessoa_df) com os setores censitários usando o campo `CD_SETOR` para comparação.

Após a união, geramos pontos aleatórios para cada pessoa da raça/cor e salvamos em um arquivo geopackage que será ordenado pelo campo `quadkey` no final do processo.

In [9]:
# lista os arquivos dos estados dentro da pasta de setores sensitários usando glob
setores_sensitarios_files = glob(path.join(setor_sensitario_folder, "*.gpkg"))
setores_sensitarios_files = filtra_arquvios_sigla_estados(setores_sensitarios_files, states)


bar_files = tqdm(
    setores_sensitarios_files,
    desc="Processando arquivos",
    unit="arquivo",
    total=len(setores_sensitarios_files),
    position=0,
    leave=True,
)
files_bar_log = tqdm(total=0, position=1, bar_format="{desc}")
stop_iteration = False
for setor_censitario_path in bar_files:
    sigla_uf = path.basename(setor_censitario_path).split("_")[0].lower()
    if sigla_uf.upper() not in states:
        continue
    files_bar_log.set_description_str(f"Processando {sigla_uf}")
    setor_censitario_gdf = gpd.read_file(setor_censitario_path)
    # Remove colunas que não são necessárias
    setor_censitario_gdf.drop(
        columns=[c for c in setor_censitario_gdf.columns if c not in setor_keep_fields],
        inplace=True,
    )
    # Renomeia as colunas
    setor_censitario_gdf.rename(columns=setor_fiels, inplace=True)
    # Fazendo a união das tabelas
    joined_gdf = setor_censitario_gdf.merge(pessoa_df, on="CD_SETOR", how="left")
    # limpa a memória do setor censitário
    del setor_censitario_gdf
    crs = joined_gdf.crs
    output_file = path.join(rand_folder, f"{sigla_uf}_pontos_pessoas.gpkg")
    makedirs(path.dirname(output_file), exist_ok=True)
    cols_to_keep = [col for col in joined_gdf.columns if col not in ["geometry"]]

    setor_bar_log = tqdm(total=0, position=3, bar_format="{desc}")
    # criando os pontos aleatórios do estado
    for idx, row in tqdm(
        joined_gdf.iterrows(),
        desc="Processando setores",
        unit="setor",
        total=joined_gdf.shape[0],
        position=2,
    ):
        setor_bar_log.set_description_str(f"Setor {row['CD_SETOR']}")
        raca_bar_log = tqdm(total=0, position=5, bar_format="{desc}")
        # Barra de progresso para raças (loop interno)
        for race, race_label in race_fields.items():
            qt_pessoas = row[race]
            if pd.isnull(qt_pessoas) or qt_pessoas == 0:
                continue
            qt_pessoas = qt_pessoas

            # Gera pontos aleatórios para a raça atual
            points = generate_random_points(row.geometry, qt_pessoas)

            # Cria o GeoDataFrame para os pontos
            points_gdf = gpd.GeoDataFrame(
                {
                    "geometry": points,
                    "raca_cor": [race_label] * len(points),
                },
                crs=crs,
            )

            # Adiciona atributos da linha ao GeoDataFrame
            for col in cols_to_keep:
                points_gdf[col] = row[col]

            # Calcula quadkey e outros atributos
            points_gdf["tile_x"], points_gdf["tile_y"], points_gdf["quadkey"] = zip(
                *[
                    (
                        *merc.MetersToTile(*merc.LatLonToMeters(p.y, p.x), 21),
                        merc.QuadTree(
                            *merc.MetersToTile(*merc.LatLonToMeters(p.y, p.x), 21), 21
                        ),
                    )
                    for p in points
                ]
            )
            # reordena as colunas
            points_gdf = points_gdf[points_cols]

            # Exporta os dados para o arquivo GPKG
            mode = "a" if path.exists(output_file) else "w"
            points_gdf.to_file(output_file, driver="GPKG", mode=mode)
            na_data = points_gdf[points_gdf["pessoas_indigenas"] == "<NA>"]
            if not na_data.empty:
                print(f"Setor {row['CD_SETOR']} tem dados faltantes")
                display(na_data)
                stop_iteration = True
                break
            # limpa a memória dos pontos
            del points_gdf
        if stop_iteration:
            break
    if  stop_iteration:
        break

    # limpa a memória do setor censitário
    print(f"Arquivo do estado {sigla_uf} processado")
    del joined_gdf

    # print("Ordenando o arquivo")
    # raca_cor_gdf = gpd.read_file(output_file)
    # raca_cor_gdf.sort_values(by="quadkey", inplace=True)
    # raca_cor_gdf.to_file(output_file, driver="GPKG", mode="w")
    # print(f"Arquivo do estado {sigla_uf} ordenado")
    # # limpa a memória do raca_cor_gdf
    # del raca_cor_gdf

Processando arquivos:   0%|                                              | 0/5 [00:00<?, ?arquivo/s]

































Pardos



















Pardos



















Pardos



















Pardos



















Pardos























Pardos



















Pardos



















Pardos



















Pardos



















Pardos



















Pardos



















Pardos



















Pardos























Indígenas



















Pardos



















Pardos


























Pardos



















Pardos



















Pardos



















Pardos























Indígenas



















Pardos



















Pardos



















Pardos



















Pardos



















Pardos



















Pardos



















Pardos



















Pardos



















Pardos



















Pardos


























Pardos



















Pardos



















Pardos



















Pardos



















Pardos





















DataSourceError: Failed to commit transaction